In [28]:
import pandas as pd
import numpy as np
import datetime
#from flask_sqlalchemy import SQLAlchemy
import sqlalchemy
from sqlalchemy.engine import url
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [29]:
# pyodbc stuff for MS SQL Server Express
driver='{SQL Server}'
server='XX.XX.XX.XX'
database='CDR_DM'
username = 'DADevUser'
password = 'XXXXXX'
trusted_connection='yes'
# pyodbc connection string
connection_string = f'DRIVER={driver};SERVER={server};'
connection_string += f'DATABASE={database};'
connection_string += f'UID={username};'
connection_string += f'PWD={password};'
#connection_string += f'TRUSTED_CONNECTION={trusted_connection}'
# create sqlalchemy engine connection URL
connection_url = url.URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = sqlalchemy.create_engine(connection_url)

In [30]:
connection = engine.connect()
metadata = sqlalchemy.MetaData()
census = sqlalchemy.Table('ML_Centene_IO_Data_Train', metadata, autoload=True, autoload_with=engine)

In [31]:
query= sqlalchemy.select([census.columns.UCRN,
census.columns.BillType,
census.columns.Header_PatientAge,
census.columns.Header_LOS,
census.columns.Header_PatientGender,
census.columns.PD_CCSRCat,
census.columns.SD1_CCSRCat,
census.columns.SD2_CCSRCat,
census.columns.SD3_CCSRCat,
census.columns.SD4_CCSRCat,
census.columns.SD5_CCSRCat,
census.columns.DischargeStatus,
census.columns.PayerResponsibilitySeqCode,
census.columns.Header_PriorAuthorizationPresent,
census.columns.Header_IsCharged,
census.columns.Header_IsAllowed,
census.columns.Header_IsPaid,
census.columns.Header_ConditionCodes1,
census.columns.Header_OccurrenceCodes1,
census.columns.Header_ValueCode1,
census.columns.Line_HCPCS_CPTCode,
census.columns.Line_IsModifierAvailable,
census.columns.Header_Quantity,
census.columns.Line_LOS,
census.columns.PlaceOfService,
census.columns.Line_IsCharged,
census.columns.Line_IsAllowed,
census.columns.Line_IsPaid,
census.columns.IsHcpcsAvlOnOthLineWtSameDtWithMod,
census.columns.IsHcpcsAvlOnOthLineWtSameDtWoMod,
census.columns.IsHcpcsAvlOnOthLineDiffDtWithMod,
census.columns.IsHcpcsAvlOnOthLineDiffDtWoMod,
census.columns.OutCome])
# .where(census.columns.Outcome in carclist)

In [33]:
#ResultProxy = connection.execute(query.limit(15000))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [34]:
pd.set_option('display.max_columns', 152)
pd.set_option('display.max_rows', 130)
pd.set_option('display.width', 200000)

df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
#df_Org = df[['UCRN','Outcome']]
df.head()

,UCRN,BillType,Header_PatientAge,Header_LOS,Header_PatientGender,Header_PrincipalDiagnosisCode,Header_SD1,Header_SD2,Header_SD3,Header_SD4,Header_SD5,Header_PriorAuthorizationPresent,Header_IsCharged,Header_IsAllowed,Header_IsPaid,Header_ConditionCodes1,Header_OccurrenceCodes1,Line_HCPCS_CPTCode,Line_IsModifierAvailable,Header_Quantity,Line_LOS,Line_IsCharged,Line_IsAllowed,Line_IsPaid,IsHcpcsAvlOnOthLineWtSameDtWithMod,IsHcpcsAvlOnOthLineWtSameDtWoMod,IsHcpcsAvlOnOthLineDiffDtWithMod,IsHcpcsAvlOnOthLineDiffDtWoMod,OutCome
0,10-1,131,65,1,F,I219,I219,I219,I219,I219,I219,0,1,0,0,0,11,93798,0,1,1,1,0,0,False,False,False,False,55
1,1000050-2,131,14,1,F,J029,J029,J029,J029,J029,J029,0,1,0,1,0,11,87804,1,1,1,1,0,0,False,True,False,False,4
2,1000089-4,131,38,1,F,O4703,O09523,Z3A34,O4703,O4703,O4703,0,1,0,1,0,11,59025,0,1,1,1,0,0,True,True,False,False,5
3,100035-1,131,16,1,F,Z00129,R062,Z23,J301,Z00129,Z00129,0,1,0,1,0,11,94664,0,1,1,1,0,0,True,True,False,False,234
4,1000408-4,131,5,1,F,J069,J069,J069,J069,J069,J069,0,1,0,1,0,11,87804,1,1,1,1,0,0,False,True,False,False,4


In [35]:

df.drop(columns=['UCRN'],inplace=True)


In [104]:
# df_Org.head()

,UCRN,Outcome
0,2217405-8,234
1,2182419-10,59
2,2182419-4,59
3,2182419-5,59
4,2182670-1,59


In [36]:
df = df.replace({False: 0, True:1})

In [8]:
# df.isnull().sum()

In [37]:
df.head(5)

,BillType,Header_PatientAge,Header_LOS,Header_PatientGender,Header_PrincipalDiagnosisCode,Header_SD1,Header_SD2,Header_SD3,Header_SD4,Header_SD5,Header_PriorAuthorizationPresent,Header_IsCharged,Header_IsAllowed,Header_IsPaid,Header_ConditionCodes1,Header_OccurrenceCodes1,Line_HCPCS_CPTCode,Line_IsModifierAvailable,Header_Quantity,Line_LOS,Line_IsCharged,Line_IsAllowed,Line_IsPaid,IsHcpcsAvlOnOthLineWtSameDtWithMod,IsHcpcsAvlOnOthLineWtSameDtWoMod,IsHcpcsAvlOnOthLineDiffDtWithMod,IsHcpcsAvlOnOthLineDiffDtWoMod,OutCome
0,131,65,1,F,I219,I219,I219,I219,I219,I219,0,1,0,0,0,11,93798,0,1,1,1,0,0,0,0,0,0,55
1,131,14,1,F,J029,J029,J029,J029,J029,J029,0,1,0,1,0,11,87804,1,1,1,1,0,0,0,1,0,0,4
2,131,38,1,F,O4703,O09523,Z3A34,O4703,O4703,O4703,0,1,0,1,0,11,59025,0,1,1,1,0,0,1,1,0,0,5
3,131,16,1,F,Z00129,R062,Z23,J301,Z00129,Z00129,0,1,0,1,0,11,94664,0,1,1,1,0,0,1,1,0,0,234
4,131,5,1,F,J069,J069,J069,J069,J069,J069,0,1,0,1,0,11,87804,1,1,1,1,0,0,0,1,0,0,4


In [18]:
df.shape

(15000, 18)

In [39]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['OutCome']),
                                                 df['OutCome'],
                                                 test_size=0.2,
                                                random_state=42)

In [40]:
#X_train.shape
X_train.head()

,BillType,Header_PatientAge,Header_LOS,Header_PatientGender,Header_PrincipalDiagnosisCode,Header_SD1,Header_SD2,Header_SD3,Header_SD4,Header_SD5,Header_PriorAuthorizationPresent,Header_IsCharged,Header_IsAllowed,Header_IsPaid,Header_ConditionCodes1,Header_OccurrenceCodes1,Line_HCPCS_CPTCode,Line_IsModifierAvailable,Header_Quantity,Line_LOS,Line_IsCharged,Line_IsAllowed,Line_IsPaid,IsHcpcsAvlOnOthLineWtSameDtWithMod,IsHcpcsAvlOnOthLineWtSameDtWoMod,IsHcpcsAvlOnOthLineDiffDtWithMod,IsHcpcsAvlOnOthLineDiffDtWoMod
9839,131,61,14,M,S31104A,S31104A,S31104A,S31104A,S31104A,S31104A,0,1,1,1,0,11,0272,0,1,1,1,0,0,0,1,0,1
9680,131,86,3,F,R531,K5900,N390,I10,E785,M48061,0,1,0,1,0,"11,18,,,,,,,,,,",99285,0,1,1,1,0,0,0,1,1,1
7093,131,39,1,F,R109,K589,Z9049,E860,N83202,N83201,0,1,1,1,0,11,J7050,0,1,1,1,0,0,1,1,0,0
11293,131,35,1,F,O0090,N888,N808,K661,L409,G43909,0,1,0,1,0,"11,18,,,,,,,,,,",81001,0,1,1,1,1,1,1,1,0,0
820,131,85,15,F,R531,R6889,R531,R531,R531,R531,0,1,1,1,0,11,97110,1,2,1,1,1,1,0,0,1,0


In [41]:
# one hot encoding
trf1 = ColumnTransformer([
    ('ohe_attr',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,3,4,5,6,7,8,9,14,15,16])
],remainder='passthrough')

In [42]:
# train the model
trf2 = RandomForestClassifier()

In [43]:
# Create Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2)
])

In [44]:
# train
pipe.fit(X_train,y_train)

print("Training Completed")


Training Completed


In [45]:
# Predict
rf_pred = pipe.predict(X_test)

print("Random Forest Accuracy: " , accuracy_score(rf_pred, y_test) * 100, "%")


Random Forest Accuracy:  91.0 %


In [46]:
# export 
import pickle
pickle.dump(pipe,open('Ovrpay_IO_Model_rf.pkl','wb'))